In [7]:
from dotenv import load_dotenv
import os

load_dotenv()

from typing import Annotated
from typing_extensions import TypedDict
from langchain.chat_models import init_chat_model
from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages


In [13]:

llm = init_chat_model("google_genai:gemini-2.5-flash")
# llm.invoke("eho is father of nation?") # its working


class State(TypedDict):
    messages: Annotated[list, add_messages]

def chatbot(state: State) -> State:
    return { "messages": [llm.invoke(state["messages"])]}


builder = StateGraph(State)

builder.add_node("chatbot_node", chatbot)

builder.add_edge(START, "chatbot_node")
builder.add_edge("chatbot_node",END)

graph= builder.compile()


In [14]:
message={"role":"user", "content":"Who lands first on moon?"}

response=graph.invoke({"messages":[message]})

response["messages"]

[HumanMessage(content='Who lands first on moon?', additional_kwargs={}, response_metadata={}, id='558f914c-27f7-423a-af79-c736b067f645'),
 AIMessage(content='**Neil Armstrong** was the first person to land on the Moon.\n\nHe stepped out of the Apollo 11 lunar module (named "Eagle") on July 20, 1969. He was followed shortly after by Buzz Aldrin.', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.5-flash', 'safety_ratings': [], 'grounding_metadata': {}, 'model_provider': 'google_genai'}, id='lc_run--dfaef30c-14dd-4cdb-a46a-0217704f1bef-0', usage_metadata={'input_tokens': 7, 'output_tokens': 291, 'total_tokens': 298, 'input_token_details': {'cache_read': 0}, 'output_token_details': {'reasoning': 238}})]

In [19]:
# Loop he context bases inforamtion


state=None
while True:
    in_msg=input("You:")

    if in_msg.lower() in ['quit','exit']:
        break

    if state is None:
        state: State = {
            "messages": [{"role": "user", "content": in_msg}]
        }
    else:
        state["messages"].append({"role":"user", "content":in_msg})

    state=graph.invoke(state)
    print("Bot: ", state["messages"][-1].content )
    


Bot:  I cannot provide real-time stock prices directly, as the information I have is not live and constantly updated. Stock prices for Amazon (AMZN) change minute by minute during market hours.

To get the *absolute latest* price for Amazon stock, you should check a reliable financial website or platform, such as:

*   **Google Finance** (just search "AMZN stock" on Google)
*   **Yahoo Finance** (finance.yahoo.com)
*   **Bloomberg**
*   **Reuters**
*   Your personal **brokerage account** (e.g., Fidelity, Schwab, E*TRADE)

These sources will give you the most current trading data, often with a slight delay depending on the service.
